In [5]:
import requests
from urllib.parse import urlencode
import os

In [ ]:
BASE_URL = "https://api.adsabs.harvard.edu/v1/search"
tardis_bibcode = "2014MNRAS.440..387K" 
tardis_articles = []
query = {
    "q": f'bibcode:"{tardis_bibcode}"',
    "fl": "title, year, author, bibstem, citation, bibcode"
}

encoded_query = urlencode(query)
all_cited_articles = []
not_refereed = []
results = requests.get(f"{BASE_URL}/query?{encoded_query}", 
                    headers={'Authorization': 'Bearer ' + os.environ['NASA_ADS_TOKEN']})
cited_bib_codes = results.json()['response']['docs'][0]['citation']
all_cited_articles.append(results.json()['response']['docs'][0])
for single_bib_code in cited_bib_codes:
    start = 0
    numFound = None
    while numFound == None or start < numFound:
        query = {
            "q": f'bibcode:"{single_bib_code}"',
            "fl": "title, author, bibstem, year, property",
            "sort": "year desc",
            "start": start
        }
        encoded_query_bib = urlencode(query)

        bib_results = requests.get(f"{BASE_URL}/query?{encoded_query_bib}", 
                            headers={'Authorization': 'Bearer ' + os.environ['NASA_ADS_TOKEN']})
        numFound = bib_results.json()['response']['numFound']
        articles = bib_results.json()['response']['docs']
        fetched_rows = len(articles)
        start += fetched_rows
        for entry in articles:
            entry['bibcode'] = single_bib_code
            not_refereed.append(entry)
            if 'REFEREED' in entry['property']:
                entry['bibcode'] = single_bib_code
                all_cited_articles.append(entry)

In [ ]:
all_cited_articles.sort(key=lambda x: x['year'], reverse=True)

In [ ]:
def format_citation(entry):
    authors = entry['author']
    if len(authors) > 3:
        author_str = ', '.join(authors[:3]) + ', et al.'
    else:
        author_str = ', '.join(authors)
    citation = f"**{author_str}** {entry['year']}, {entry['bibstem'][0]}, *\"{entry['title']}\"* `(ADS Link) <https://ui.adsabs.harvard.edu/abs/{entry['bibcode']}>`__"
    return citation

In [ ]:
with open('resource_papers.rst', 'w') as rst_file:
    rst_file.write('###################\n')
    rst_file.write('Papers Using TARDIS\n')
    rst_file.write('###################\n\n')
    for article in all_cited_articles:
        formatted_citation = format_citation(article)
        rst_file.write(formatted_citation + '\n\n')